# SEIR model example


\begin{align}
\dot{S} & = - \alpha\beta\frac{SI}{N}\\
\dot{E} & =  \alpha\beta\frac{SI}{N} - E\frac{1}{\tau_{EI}}\\
\dot{I} & =  E\frac{1}{\tau_{EI}} - I\frac{1}{\tau_{IR}}\\
\dot{R} & =  I\frac{1}{\tau_{IR}} \\
\end{align}

# SEIR model example
\begin{align}
\dot{S} & = S_f - \alpha\beta\frac{SI}{N} + \rho_{RS} R + \phi_S\\
\dot{E} & = E_f + \alpha\beta\frac{SI}{N} - E\frac{1}{\tau_{EI}} + \phi_E\\
\dot{I} & = I_f + E\frac{1}{\tau_{EI}} - I\frac{1}{\tau_{IR}} + \phi_I\\
\dot{R} & = R_f + I\frac{1}{\tau_{IR}} - \rho_{IR} R + \phi_R\\
\dot{\Phi} & = \sum_{i} (\phi_{i}) \\
N & = N_0 + \Phi \\
N_0 & = S_0+E_0+I_0+R_0  \\
\end{align}

Where:  
* $S:$ Susceptible Individuals
* $E:$ Exposed Individuals
* $I:$ Infectious Individuals
* $R:$ Removed Individuals
* $\alpha:$ Average number of contacts per person per time
* $\beta:$ Probability of disease transmission in a contact between a susceptible and an infectious subject
* $N:$ Total population
* $\tau_{EI}:$ Transition time between exposed and infectious
* $\tau_{IR}:$ Transition time between infectious and recovered
* $\rho_{RS}:$ Immunity loss rate. Inverse of Immunity loss rate medium time ($\frac{1}{t_{RS}}$)  
* $\phi_S,\phi_E,\phi_I,\phi_R:$ External flux per compartment
* $\Phi$: Total external population flux 

In [ ]:
# Util libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# cv19 libraries
from cv19gm.models.seir import SEIR
import cv19gm.utils.cv19functions as cv19functions

In [ ]:
# For pop-up plots execute this code (optional)
import platform
OS = platform.system()

if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')

# Build a basic SEIR model using a configuration file

## Constant parameters

In [ ]:
# Input configuration file
config = 'cfg/SEIR.toml'

In [ ]:
# Build simulation object
model = SEIR(config = config)

In [ ]:
%%capture
# Simulate (solve ODE)
model.solve()

In [ ]:
# Show results
model.results

In [ ]:
# Plot matplotlib
plt.plot(model.t,model.S,label='S')
plt.plot(model.t,model.E,label='E')
plt.plot(model.t,model.I,label='I')
plt.plot(model.t,model.R,label='R')
plt.xlim(0,300)
plt.legend(loc=0)
plt.show()

In [ ]:
# Plot with pandas
ax = model.results.plot(x='t',y=['S','E','I','R'])
ax.set_xlim(0,300)
ax.set_title('SEIR simulation plot')

## Variable parameters
Create a function with time dependent parameters. Check covid19functions notebook for more information on this.

In [ ]:
config2 = 'cfg/SEIR_v2.toml'

In [ ]:
model2 = SEIR(config = config2)

In [ ]:
%%capture
model2.solve()

In [ ]:
plt.plot(model2.t,model2.S,label='S')
plt.plot(model2.t,model2.E,label='E')
plt.plot(model2.t,model2.I,label='I')
plt.plot(model2.t,model2.R,label='R')
plt.xlim(0,300)
plt.legend(loc=0)
plt.show()

# SEIR desde cmd

In [ ]:
beta = 0.2
population = 10000
I = 100

In [ ]:
model = SEIR(config = config)

In [ ]:
model.solve()

### Variable parameters

In [ ]:
t = np.linspace(0,6,1000)

In [ ]:
beta = lambda t: np.sin(t)

In [ ]:
plt.plot(t,beta(t))

In [ ]:
model = SEIR(config = config,beta=beta)

In [ ]:
model.solve()

In [ ]:
plt.plot(model.t,model.I)
plt.title('Active Infected')
plt.xlim(0,50)
plt.show()

In [ ]:
beta2 = cv19functions.events(days=[[0,10],[10,60],[60,100]],values=[0.12,0.2,0.3],default = 0.1)

In [ ]:
t = np.linspace(0,200,10000)

In [ ]:
plt.plot(t,beta2(t))

In [ ]:
model2 = SEIR(config = config,beta=beta2)

In [ ]:
model2.solve()

In [ ]:
plt.plot(model2.t,model2.I)

# Study

In [ ]:
# Input configuration file
config = 'cfg/SEIR.toml'

In [ ]:
# Build simulation object
model1 = SEIR(config = config, I_d_det = 600, population = 1000000)
model2 = SEIR(config = config, I_d_det = 600, population = 10000000)

In [ ]:
%%capture
# Simulate (solve ODE)
model1.solve()
model2.solve()

In [ ]:
model1.I_d[0]

In [ ]:
# Plot matplotlib
#plt.plot(model.t,model1.S,label='S')
#plt.plot(model.t,model1.E,label='E')
plt.plot(model.t,model1.I_d,label='I_1')
plt.plot(model.t,model2.I_d,label='I_2')
plt.xlim(0,300)
plt.yscale('log')
plt.legend(loc=0)
plt.show()